# 1. Introduction to the Perceptron

Welcome! This notebook will guide you through building your very first deep learning algorithm, the **Perceptron**, from scratch.

Perceptrons were one of the first algorithms discovered in the field of AI. Their significance was that they raised the hopes and expectations for the field of neural networks. A Perceptron is a machine learning algorithm that uses a single node (or "neuron") to predict a class label for a row of data. It's the simplest possible type of neural network.



### The Four Components of a Perceptron

A Perceptron has four main parts. Think of it as a simple decision-making process:

1.  **Input Values ($x$):** This is your data. For example, in our case, we'll have two input values, `x1` and `x2`.
2.  **Weights ($w$) and Bias ($b$):**
    * **Weights:** Each input value has a corresponding "weight". A weight represents how important that input is for the final decision. A bigger weight means that input matters more.
    * **Bias:** The bias is an extra value that helps the model make a decision.
    * *Learning* is simply the process of finding the perfect set of weights and bias.
3.  **Net Sum ($z$):** This is the first calculation the Perceptron does. It multiplies each input by its weight, sums them all up, and then adds the bias.

    $z = (x_1 \cdot w_1) + (x_2 \cdot w_2) + ... + b$

4.  **Activation Function:** This is the final step. The Perceptron takes the `Net Sum` ($z$) and passes it through a simple function to make a final, binary decision (0 or 1).
    * The simplest activation function is a **step function**:
    * If the `Net Sum` ($z$) is greater than 0, predict **1**.
    * Otherwise, predict **0**.

# 2. Setup and Creating the Dataset

First, let's import the libraries we'll need.

* `numpy`: For numerical operations.
* `pandas`: To easily look at our data in a table.
* `matplotlib`: For plotting our data.
* `sklearn.datasets.make_classification`: A handy function to create a simple, fake dataset for us to play with.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

### Creating the dataset

Here we generate a simulated dataset using scikit-learn's `make_classification` function. This creates a dataset with 2 input features and binary target labels.

We're asking for:
* `n_samples = 20`: 20 rows of data.
* `n_features = 2`: 2 input features (we'll call them `x1` and `x2`).
* `n_informative = 1`: Only 1 of the features is *actually* useful for telling the classes apart.
* `n_redundant = 0`: No extra, useless features.
* `n_clusters_per_class = 1`: Each class (0 and 1) is just one single "blob" of data.
* `random_state = 1`: This ensures that if you run this code, you get the *exact same* "random" data that I do.

In [ ]:
features, targets = make_classification(n_samples = 20,
                                      n_features = 2,
                                      n_informative = 1,
                                      n_redundant = 0,
                                      n_clusters_per_class = 1,
                                      random_state = 1)

Let's put this data into a `pandas` DataFrame to see what it looks like.

In [ ]:
df = pd.DataFrame(data=features, columns=['x1', 'x2'])
df['targets'] = targets
df.head()

Let's check the `shape` of our data. This tells us (rows, columns).

In [ ]:
features.shape

In [ ]:
targets.shape

`np.bincount` is a quick way to count how many samples of each class we have.

In [ ]:
np.bincount(targets)

This confirms we have 10 samples of Class 0 and 10 samples of Class 1. A nice, balanced dataset.

# 3. Visualizing the dataset

We can visualize the dataset by plotting the two input features colored by the target class. This gives us a sense of how linearly separable the data is.

A **linearly separable** dataset is one where you can draw a single straight line to separate the two classes. A Perceptron can *only* solve problems that are linearly separable.

As we can see below, our data isn't *perfectly* separable, but there is an approximate linear decision boundary. Let's see if our Perceptron can find it.

In [ ]:
plt.plot(
    features[targets == 0, 0],
    features[targets == 0, 1],
    marker = 'P',
    markersize = 10,
    linestyle = '',
    label = 'Class 0'
    )
plt.plot(
    features[targets == 1, 0],
    features[targets == 1, 1],
    marker = '^',
    markersize = 10,
    linestyle = '',
    label = 'Class 1')
plt.legend(loc = 2)
plt.xlim(-2, 2)
plt.ylim(-2, 2)
plt.xlabel("Feature $x_1$", fontsize=12)
plt.ylabel("Feature $x_2$", fontsize=12)
plt.grid()
plt.show()

# 4. Implementing a Perceptron

We can now implement the perceptron algorithm in Python. We will do this by creating a `Perceptron` class.

First, let's define the `__init__` method. This method is called whenever we create a new Perceptron object. It's used to set up our initial variables: `weights` and `bias`.

We'll initialize all weights to `0.0` and the bias to `0`.

In [ ]:
class Perceptron:
    def __init__(self, num_features):
        self.num_features = num_features
        self.weights = [0.0 for _ in range(num_features)]
        self.bias = 0

ppn = Perceptron(num_features = 2)
ppn.weights

In [ ]:
ppn.bias

### Implementing the forward function

The **forward pass** is how the Perceptron makes a prediction.

The `forward` function will compute the weighted sum ($z$) and then use the activation function (the step function) to return a binary prediction (0 or 1).

In [ ]:
class Perceptron:
    def __init__(self, num_features):
        self.num_features = num_features
        self.weights = [0.0 for _ in range(num_features)]
        self.bias = 0

    def forward(self, x):
        weighted_sum_z = self.bias
        for i, _ in enumerate(self.weights):
            weighted_sum_z += x[i] * self.weights[i]

        if weighted_sum_z > 0:
            prediction = 1
        else:
            prediction = 0

        return prediction

ppn = Perceptron(num_features = 2) # Re-initialize with 2 features
x = [1.23, 2.13]
ppn.forward(x)

The prediction in the above case is 0 because the weights and bias of the perceptron are still 0. It has not learned anything yet.

### Updating the weights (The Learning Rule)

This is the most important part! The `update` method is where the *learning* happens. It takes a single training example (`x`, `y_true`), compares the `prediction` to the `y_true` label, and updates the weights and bias accordingly.

The update rule is:
1.  Calculate the `error = y_true - prediction`.
    * If the prediction is correct (0 vs 0 or 1 vs 1), the `error` will be **0**.
    * If it predicts 0 but should be 1, the `error` will be **1**.
    * If it predicts 1 but should be 0, the `error` will be **-1**.
2.  Update the bias: `self.bias += error`
3.  Update the weights: `self.weights[i] += error * x[i]`

This means that if the error is 0, **no updates happen**. The model only learns when it makes a mistake.

In [ ]:
class Perceptron:
    def __init__(self, num_features):
        self.num_features = num_features
        self.weights = [0.0 for _ in range(num_features)]
        self.bias = 0

    def forward(self, x):
        weighted_sum_z = self.bias
        for i, _ in enumerate(self.weights):
            weighted_sum_z += x[i] * self.weights[i]

        if weighted_sum_z > 0:
            prediction = 1
        else:
            prediction = 0

        return prediction

    def update(self, x, y_true):
        prediction = self.forward(x)
        error = y_true - prediction

        self.bias += error
        for i, _ in enumerate(self.weights):
            self.weights[i] += error * x[i]

        return error

ppn = Perceptron(num_features = 2)
x = [1.1, 2.1]
ppn.update(x, y_true = 1)

Let's check what happened to the weights and bias. Since the prediction was 0 (all weights were 0), and the true label was 1, the `error` was `1 - 0 = 1`.

The new bias should be `0 + 1 = 1`.
The new weights should be `[0 + (1 * 1.1), 0 + (1 * 2.1)]` = `[1.1, 2.1]`.

In [ ]:
print("Model parameters:")
print("Weights:", ppn.weights)
print("Bias:", ppn.bias)

# 5. Training the model

Now we can write a `train` function to automate the training process.

An **epoch** is one full pass through the entire training dataset. We will train the perceptron by iterating through the dataset for several epochs. In each epoch, we loop through all the `(x, y)` examples and call the `update` method for each one.

We'll also count the number of errors in each epoch. As the model learns, we should see the number of errors go down.

In [ ]:
def train(model, X_train, y_train, epochs):
    # Training Loop
    for epoch in range(epochs):
        error_count = 0

        for x, y in zip(X_train, y_train):
            error = model.update(x, y)
            error_count += abs(error)

        print(f"Epoch {epoch + 1} errors {error_count}")

Let's create a new Perceptron and train it for 5 epochs on our `features` and `targets`.

In [ ]:
ppn = Perceptron(num_features = 2)
train(ppn, features, targets, epochs = 5)

Look at that! After just 2 epochs, the model made **0 errors**. This means it has found a set of weights and bias that perfectly separates the training data. The Perceptron algorithm is guaranteed to find a solution *if* one exists (i.e., if the data is linearly separable).

# 6. Evaluating the model

We can evaluate the model on the training data by comparing its predictions to the true labels and computing the prediction accuracy.

In [ ]:
def compute_accuracy(model, features, targets):
    correct = 0.0

    for x, y in zip(features, targets):
        prediction = model.forward(x)
        correct += int(prediction == y)

    return correct / len(targets)

In [ ]:
train_acc = compute_accuracy(ppn, features, targets)
print("Model Accuracy:", train_acc)

An accuracy of 1.0 means 100% correct. Perfect!

# 7. Plotting the Decision Boundary

Finally, we can visualize the perceptron's linear decision boundary. The line that the Perceptron learns is where the `weighted_sum_z` is exactly 0.

$z = (w_1 \cdot x_1) + (w_2 \cdot x_2) + b = 0$

We can solve this for $x_2$ to get the equation of a line:

$x_2 = \frac{-(w_1 \cdot x_1) - b}{w_2}$

We can use this equation to plot the line (or "dividing hyperplane") that our model has learned.

In [ ]:
def plot_boundary(model):

    w1, w2 = model.weights[0], model.weights[1]
    b = model.bias

    # z = w1*x1 + w2*x2 + b = 0
    # => x2 = (-w1*x1 - b) / w2

    x1_min = -20
    x2_min = (-(w1 * x1_min) - b) / w2

    x1_max = 20
    x2_max = (-(w1 * x1_max) - b) / w2

    return x1_min, x1_max, x2_min, x2_max

In [ ]:
x1_min, x1_max, x2_min, x2_max = plot_boundary(ppn)

plt.plot(
    features[targets == 0, 0],
    features[targets == 0, 1],
    marker="D",
    markersize=10,
    linestyle="",
    label="Class 0",
)
plt.plot(
    features[targets == 1, 0],
    features[targets == 1, 1],
    marker="^",
    markersize=13,
    linestyle="",
    label="Class 1",
)

# Plot the decision boundary
plt.plot([x1_min, x1_max], [x2_min, x2_max], color="k")

plt.legend(loc=2)
plt.xlim([-2, 2])
plt.ylim([-2, 2])
plt.xlabel("Feature $x_1$", fontsize=12)
plt.ylabel("Feature $x_2$", fontsize=12)
plt.grid()
plt.show()

### Conclusion

And there it is! The black line is the decision boundary learned by our Perceptron. Everything on one side of the line will be predicted as Class 0, and everything on the other side will be predicted as Class 1.

This demonstrates how a simple Perceptron model can learn a linear classifier from data. While limited to linearly separable problems, the Perceptron formed the foundation for all the more complex neural networks that are used today.